In [41]:
import os

project_folder = './pytorch-peds'
os.makedirs(project_folder, exist_ok=True)

Possibly helpful: [this link](https://github.com/drabastomek/GTC/blob/master/SJ_2020/workshop/1_Setup/Setup.ipynb), and this sample dockerfile from Jordan:

```
FROM mcr.microsoft.com/azureml/base-gpu:intelmpi2018.3-cuda9.0-cudnn7-ubuntu16.04

# Install Horovod, temporarily using CUDA stubsddd 
RUN ldconfig /usr/local/cuda/lib64/stubs && \     
# Install AzureML SDK     
pip install --no-cache-dir azureml-defaults && \     
# Install PyTorch     
pip install --no-cache-dir tensorflow==2.0.0b1 tensorflow-gpu==2.0.0b1 keras==2.0.8 matplotlib==3.0.3 seaborn==0.9.0 requests==2.21.0 bs4==0.0.1 imageio==2.5.0 sklearn pandas==0.24.2 numpy==1.16.2 hickle==3.4.3 && \     
# Install Horovod     
pip install --no-cache-dir horovod==0.13.5 && \     ldconfig
```

In [42]:
import shutil

shutil.copy('script.py', project_folder)

'./pytorch-peds/script.py'

In [43]:
#import sys
#sys.path.insert(0,'./')
# Install pycocotools
# !git clone https://github.com/cocodataset/cocoapi.git
# !ls
# %cd cocoapi/PythonAPI
# !ls
# !pip install cython
# !pip install numpy
# !python setup.py build_ext install

In [44]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.85


In [45]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [46]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: gopalv-ws
Azure region: westus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: aifxdemo


In [71]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-02-14T22:38:55.493000+00:00', 'errors': None, 'creationTime': '2020-02-14T22:38:17.474022+00:00', 'modifiedTime': '2020-02-14T22:39:03.552424+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [72]:
import urllib.request
 
url = 'https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip'
 
urllib.request.urlretrieve(url, './test.zip')

('./test.zip', <http.client.HTTPMessage at 0x141706280>)

In [73]:
from zipfile import ZipFile

zip = ZipFile(file='./test.zip')
zip.extractall()

In [74]:
!ls PennFudanPed/

Annotation            PedMasks              readme.txt
PNGImages             added-object-list.txt


In [75]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

workspaceblobstore AzureBlob gopalvws3790775563 azureml-blobstore-e47496c6-9688-4277-a05b-ceb722514b9d


In [76]:
ds.upload('./PennFudanPed', target_path='data', overwrite=False)

Uploading an estimated of 512 files
Target already exists. Skipping upload for data/added-object-list.txt
Target already exists. Skipping upload for data/readme.txt
Target already exists. Skipping upload for data/PedMasks/PennPed00048_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00049_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00072_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00073_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00001_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00093_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00092_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00076_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00077_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00005_mask.png
Target already exists. Skipping upload for

Target already exists. Skipping upload for data/PedMasks/FudanPed00032_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00033_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00008_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00009_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00041_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00040_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00036_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00037_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00045_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00044_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00071_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00070_mask.png
Target already exists. Skipping upload for dat

Target already exists. Skipping upload for data/Annotation/PennPed00071.txt
Target already exists. Skipping upload for data/Annotation/PennPed00067.txt
Target already exists. Skipping upload for data/Annotation/PennPed00073.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00074.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00060.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00048.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00049.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00061.txt
Target already exists. Skipping upload for data/Annotation/PennPed00072.txt
Target already exists. Skipping upload for data/Annotation/PennPed00066.txt
Target already exists. Skipping upload for data/Annotation/PennPed00089.txt
Target already exists. Skipping upload for data/Annotation/PennPed00062.txt
Target already exists. Skipping upload for data/Annotation/PennPed00076.txt
Target 

Target already exists. Skipping upload for data/Annotation/FudanPed00027.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00026.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00032.txt
Target already exists. Skipping upload for data/Annotation/PennPed00035.txt
Target already exists. Skipping upload for data/Annotation/PennPed00021.txt
Target already exists. Skipping upload for data/Annotation/PennPed00009.txt
Target already exists. Skipping upload for data/Annotation/PennPed00037.txt
Target already exists. Skipping upload for data/Annotation/PennPed00023.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00018.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00024.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00030.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00031.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00025.txt
Targ

Target already exists. Skipping upload for data/PNGImages/FudanPed00022.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00036.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00032.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00026.png
Target already exists. Skipping upload for data/PNGImages/PennPed00021.png
Target already exists. Skipping upload for data/PNGImages/PennPed00035.png
Target already exists. Skipping upload for data/PNGImages/PennPed00009.png
Target already exists. Skipping upload for data/PNGImages/PennPed00008.png
Target already exists. Skipping upload for data/PNGImages/PennPed00034.png
Target already exists. Skipping upload for data/PNGImages/PennPed00020.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00027.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00033.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00025.png
Target already exi

$AZUREML_DATAREFERENCE_a5a6931f12d34ac183b159b87183ff57

In [77]:
from azureml.core import Dataset
# create a FileDataset pointing to files in 'animals' folder and its subfolders recursively
datastore_paths = [(ds, 'data')]
penn_ds = Dataset.File.from_files(path=datastore_paths)
penn_ds.register(workspace=ws,
                 name='penn_ds',
                 description='Penn Fudan pedestrian data')

{
  "source": [
    "('workspaceblobstore', 'data')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "1cd2dd50-ae6d-44f9-81e2-f0044c862efe",
    "name": "penn_ds",
    "version": 1,
    "description": "Penn Fudan pedestrian data",
    "workspace": "Workspace.create(name='gopalv-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='aifxdemo')"
  }
}

In [78]:
# dataset_name = 'penn_ds'

# # Get a dataset by name
# penn1_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [79]:
# from PIL import Image
# import os
# import glob


# with penn_ds.mount() as mount_context:
#     # list top level mounted files and folders in the dataset
#     imgs_path =(os.path.join(mount_context.mount_point, 'PNGImages'))
#     print(f'imgs_path is {imgs_path}')
#     print(list(sorted(os.listdir(imgs_path))))
#     Image.open(os.path.join(mount_context.mount_point, 'PNGImages/PennPed00036.png'))
# penn1_ds.as_named_input('test').as_mount()

In [80]:
# PennFudanDataset(penn_ds)

In [81]:
# !ls
# %cd ..
# !ls

In [82]:
!git clone https://github.com/pytorch/vision.git

!git checkout v0.3.0

%cd vision
!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../
%cd ..

fatal: destination path 'vision' already exists and is not an empty directory.
error: pathspec 'v0.3.0' did not match any file(s) known to git
/Users/gkv/MS/pytorch-object/vision
/Users/gkv/MS/pytorch-object


In [83]:
import shutil

os.listdir()

files_to_copy = ['utils', 'transforms', 'coco_eval', 'engine', 'coco_utils']
for file in files_to_copy:
    shutil.copy('./vision/references/detection/'+ file + '.py', project_folder)
shutil.copy('./conda-dependencies.yml', project_folder)


'./pytorch-peds/conda-dependencies.yml'

In [84]:
from azureml.core import Experiment

experiment_name = 'pytorch-peds'
experiment = Experiment(ws, name=experiment_name)

In [85]:
from azureml.core import Environment

my_env = Environment(name='maskr-docker')
my_env.docker.enabled = True
with open("dockerfiles/Dockerfile1", "r") as f:
    dockerfile_contents_of_your_base_image=f.read()
my_env.docker.base_dockerfile=dockerfile_contents_of_your_base_image 
my_env.docker.base_image = None # 'mcr.microsoft.com/azureml/base-gpu:openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'
my_env.python.interpreter_path = '/opt/miniconda/bin/conda'
my_env.python.user_managed_dependencies = True

# from azureml.core import Environment

# my_env = Environment.from_conda_specification(name = "myenv",
#                                              file_path = "conda-dependencies.yml")

In [86]:
# from azureml.train.estimator import Estimator

# my_est = Estimator(
#     source_directory=project_folder,
#     entry_script="script.py",
#     compute_target="local",
#     use_gpu=True,
#     conda_packages=['torchvision">=0.5.0"', 'torch', 'numpy', 'cython'])

In [87]:
from azureml.train.dnn import PyTorch
from azureml.core import ScriptRunConfig

# follow pattern from here: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments#use-environments-for-training

# Add training script to run config
runconfig = ScriptRunConfig(source_directory=project_folder, script="script.py")

# Attach compute target to run config
runconfig.run_config.target = cluster_name#"local"


# Attach environment to run config
runconfig.run_config.environment = my_env

# Submit run 
run = experiment.submit(runconfig)
print(run)

Run(Experiment: pytorch-peds,
Id: pytorch-peds_1582693693_80d8cfdf,
Type: azureml.scriptrun,
Status: Starting)


In [88]:
# to get more details of your run
print(run.get_details())

{'runId': 'pytorch-peds_1582693693_80d8cfdf', 'target': 'gpu-cluster1', 'status': 'Starting', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': 'ba9ad21b-f7f1-4962-9c50-397c3a8ce7d7', 'azureml.git.repository_uri': 'https://github.com/gvashishtha/pytorch-object.git', 'mlflow.source.git.repoURL': 'https://github.com/gvashishtha/pytorch-object.git', 'azureml.git.branch': 'docker', 'mlflow.source.git.branch': 'docker', 'azureml.git.commit': 'f7f0e91222e64b65afe0e0e5f2b967829e93c341', 'mlflow.source.git.commit': 'f7f0e91222e64b65afe0e0e5f2b967829e93c341', 'azureml.git.dirty': 'False'}, 'inputDatasets': [], 'runDefinition': {'script': 'script.py', 'useAbsolutePath': False, 'arguments': [], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'gpu-cluster1', 'dataReferences': {}, 'data': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'environment': {'name': 'maskr-docker', 'version': 'Autosave_2020-02-26T

In [89]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1582693693_80d8cfdf
Web View: https://ml.azure.com/experiments/pytorch-peds/runs/pytorch-peds_1582693693_80d8cfdf?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/aifxdemo/workspaces/gopalv-ws

Streaming azureml-logs/20_image_build_log.txt

2020/02/26 05:08:30 Downloading source code...
2020/02/26 05:08:35 Finished downloading source code
2020/02/26 05:08:35 Creating Docker network: acb_default_network, driver: 'bridge'
2020/02/26 05:08:36 Successfully set up Docker network: acb_default_network
2020/02/26 05:08:36 Setting up Docker configuration...
2020/02/26 05:08:36 Successfully set up Docker configuration
2020/02/26 05:08:36 Logging in to registry: gopalvwsdebb3b12.azurecr.io
2020/02/26 05:08:38 Successfully logged into gopalvwsdebb3b12.azurecr.io
2020/02/26 05:08:38 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/02/26 05:08:38 Scanning for dependencies...
2020/02/26 05:08:39 Su


pip-20.0.2           | 1.9 MB    |            |   0% 
pip-20.0.2           | 1.9 MB    | #######8   |  78% 
pip-20.0.2           | 1.9 MB    | #########8 |  98% 
pip-20.0.2           | 1.9 MB    | ########## | 100% 

_libgcc_mutex-0.1    | 3 KB      |            |   0% 
_libgcc_mutex-0.1    | 3 KB      | ########## | 100% 

urllib3-1.25.8       | 166 KB    |            |   0% 
urllib3-1.25.8       | 166 KB    | ########## | 100% 

wheel-0.34.2         | 49 KB     |            |   0% 
wheel-0.34.2         | 49 KB     | ########## | 100% 

six-1.14.0           | 27 KB     |            |   0% 
six-1.14.0           | 27 KB     | ########## | 100% 

conda-4.8.2          | 3.0 MB    |            |   0% 
conda-4.8.2          | 3.0 MB    | #######6   |  77% 
conda-4.8.2          | 3.0 MB    | #########4 |  94% 
conda-4.8.2          | 3.0 MB    | ########## | 100% 

setuptools-45.2.0    | 656 KB    |            |   0% 
setuptools-45.2.0    | 656 KB    | ########9  |  89% 
setuptools-45.2.0    


libtiff-4.1.0        | 447 KB    |            |   0% 
libtiff-4.1.0        | 447 KB    | 3          |   4% 
libtiff-4.1.0        | 447 KB    | ########## | 100% 

cudatoolkit-10.1.243 | 347.4 MB  |            |   0% 
cudatoolkit-10.1.243 | 347.4 MB  | 2          |   2% 
cudatoolkit-10.1.243 | 347.4 MB  | 5          |   5% 
cudatoolkit-10.1.243 | 347.4 MB  | 8          |   8% 
cudatoolkit-10.1.243 | 347.4 MB  | #1         |  12% 
cudatoolkit-10.1.243 | 347.4 MB  | #4         |  15% 
cudatoolkit-10.1.243 | 347.4 MB  | #7         |  18% 
cudatoolkit-10.1.243 | 347.4 MB  | ##1        |  21% 
cudatoolkit-10.1.243 | 347.4 MB  | ##4        |  24% 
cudatoolkit-10.1.243 | 347.4 MB  | ##7        |  28% 
cudatoolkit-10.1.243 | 347.4 MB  | ###        |  31% 
cudatoolkit-10.1.243 | 347.4 MB  | ###4       |  34% 
cudatoolkit-10.1.243 | 347.4 MB  | ###7       |  38% 
cudatoolkit-10.1.243 | 347.4 MB  | ####       |  41% 
cudatoolkit-10.1.243 | 347.4 MB  | ####4      |  44% 
cudatoolkit-10.1.243 | 347

Removing intermediate container 26a9d0d8ca89
 ---> 157579da5d01
Step 6/14 : RUN git clone https://github.com/cocodataset/cocoapi.git && cd cocoapi/PythonAPI && python setup.py build_ext install
 ---> Running in ed868f50f3c9
Cloning into 'cocoapi'...
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.8
creating build/temp.linux-x86_64-3.8/pycocotools
gcc -pthread -B /opt/miniconda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/miniconda/lib/python3.8/site-packages/numpy/core/include -I../common -I/opt/miniconda/include/python3.8 -c ../common/maskApi.c -o build/temp.linux-x86_64-3.8/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function â€˜rleDecodeâ€™:
../common/maskApi.c:46:7: warning: this â€˜forâ€™ clause does not guard... [-Wmisleading-indentation]
     

Installing kiwisolver-1.1.0-cp38-cp38-manylinux1_x86_64.whl to /opt/miniconda/lib/python3.8/site-packages
Adding kiwisolver 1.1.0 to easy-install.pth file

Installed /opt/miniconda/lib/python3.8/site-packages/kiwisolver-1.1.0-py3.8-linux-x86_64.egg
Searching for cycler>=0.10
Reading https://pypi.org/simple/cycler/
Best match: cycler 0.10.0
Processing cycler-0.10.0-py2.py3-none-any.whl
Installing cycler-0.10.0-py2.py3-none-any.whl to /opt/miniconda/lib/python3.8/site-packages
Adding cycler 0.10.0 to easy-install.pth file

Installed /opt/miniconda/lib/python3.8/site-packages/cycler-0.10.0-py3.8.egg
Searching for Cython==0.29.15
Best match: Cython 0.29.15
Adding Cython 0.29.15 to easy-install.pth file
Installing cygdb script to /opt/miniconda/bin
Installing cython script to /opt/miniconda/bin
Installing cythonize script to /opt/miniconda/bin

Using /opt/miniconda/lib/python3.8/site-packages
Searching for setuptools==45.2.0.post20200210
Best match: setuptools 45.2.0.post20200210
Adding set

d8f1569ddae6: Pull complete
85386706b020: Pull complete
ee9b457b77d0: Pull complete
b398e882f414: Verifying Checksum
b398e882f414: Download complete
be4f3343ecd3: Verifying Checksum
be4f3343ecd3: Download complete
f2e1f2321196: Verifying Checksum
f2e1f2321196: Download complete
1e87b0a17e1a: Verifying Checksum
1e87b0a17e1a: Download complete
e446a9769f53: Verifying Checksum
e446a9769f53: Download complete
6aadf6f91a8d: Verifying Checksum
6aadf6f91a8d: Download complete
66d0708eeaf3: Verifying Checksum
66d0708eeaf3: Download complete
4493432064f7: Verifying Checksum
4493432064f7: Download complete
5952336ab5bb: Verifying Checksum
5952336ab5bb: Download complete
30b4effda4fd: Verifying Checksum
30b4effda4fd: Download complete
96c67a8169ce: Verifying Checksum
96c67a8169ce: Download complete
c30b3e1cc026: Verifying Checksum
c30b3e1cc026: Download complete
72123859ee47: Verifying Checksum
72123859ee47: Download complete
f884d0346b03: Verifying Checksum
f884d0346b03: Download complete
9b0110

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "ServiceError",
        "message": "AzureMLCompute job failed.\nJobPreparationFailed: Job preparation failed with a non-zero exit code; see the job preparation log file for details.\n\tInfo: Job environment preparation failed on 10.0.0.4.",
        "details": []
    },
    "correlation": {
        "operation": null,
        "request": "8e4b52bdf80492f7"
    },
    "environment": "westus2",
    "location": "westus2",
    "time": "2020-02-26T05:26:12.485729Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"ServiceError\",\n        \"message\": \"AzureMLCompute job failed.\\nJobPreparationFailed: Job preparation failed with a non-zero exit code; see the job preparation log file for details.\\n\\tInfo: Job environment preparation failed on 10.0.0.4.\",\n        \"details\": []\n    },\n    \"corr

In [90]:
from azureml.train.estimator import Estimator

script_params = {
    '--num_epochs': 10,
    '--output_dir': './outputs'
}

estimator = Estimator(source_directory=project_folder, 
                      script_params=script_params,
                      compute_target=compute_target,
                      entry_script='script.py',
                      use_gpu=True,
                      conda_dependencies_file='conda-dependencies.yml')

In [91]:
run = experiment.submit(estimator)
print(run.get_details())

TrainingException: TrainingException:
	Message: {
    "error_details": {
        "correlation": {
            "operation": "04acdb6530c8a64a9a441734c145be21",
            "request": "0e0fa31e21b64f25"
        },
        "environment": "westus2",
        "error": {
            "code": "UserError",
            "debugInfo": {
                "innerException": {
                    "errorResponse": {
                        "correlation": {
                            "operation": "04acdb6530c8a64a9a441734c145be21",
                            "request": "49e641e39e5594c9"
                        },
                        "environment": "westus2",
                        "error": {
                            "code": "ValidationError",
                            "debugInfo": {
                                "message": "Python interpreter version must be specified in the Conda file.",
                                "stackTrace": "   at Microsoft.MachineLearning.EnvironmentManagement.Ser

In [70]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1582507355_50a4f6df
Web View: https://ml.azure.com/experiments/pytorch-peds/runs/pytorch-peds_1582507355_50a4f6df?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/aifxdemo/workspaces/gopalv-ws

Streaming azureml-logs/20_image_build_log.txt

2020/02/24 01:22:52 Downloading source code...
2020/02/24 01:22:53 Finished downloading source code
2020/02/24 01:22:54 Creating Docker network: acb_default_network, driver: 'bridge'
2020/02/24 01:22:54 Successfully set up Docker network: acb_default_network
2020/02/24 01:22:54 Setting up Docker configuration...
2020/02/24 01:22:55 Successfully set up Docker configuration
2020/02/24 01:22:55 Logging in to registry: gopalvwsdebb3b12.azurecr.io
2020/02/24 01:22:56 Successfully logged into gopalvwsdebb3b12.azurecr.io
2020/02/24 01:22:56 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/02/24 01:22:56 Scanning for dependencies...
2020/02/24 01:22:57 Su

Executing transaction: ...working... done
ERROR: Could not find a version that satisfies the requirement cocoapi (from -r /azureml-environment-setup/condaenv.9ty031gd.requirements.txt (line 2)) (from versions: none)
ERROR: No matching distribution found for cocoapi (from -r /azureml-environment-setup/condaenv.9ty031gd.requirements.txt (line 2))


CondaValueError: pip returned an error

The command '/bin/sh -c ldconfig /usr/local/cuda/lib64/stubs && conda env create -p /azureml-envs/azureml_a45a0b01d0acbd550b290c3aa915e399 -f azureml-environment-setup/mutated_conda_dependencies.yml && rm -rf "$HOME/.cache/pip" && conda clean -aqy && CONDA_ROOT_DIR=$(conda info --root) && rm -rf "$CONDA_ROOT_DIR/pkgs" && find "$CONDA_ROOT_DIR" -type d -name __pycache__ -exec rm -rf {} + && ldconfig' returned a non-zero code: 1
2020/02/24 01:25:05 Container failed during run: acb_step_0. No retries remaining.
failed to run step ID: acb_step_0: exit status 1

Run ID: ccj failed after 2m13s. Error: failed d

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt",
        "details": []
    },
    "correlation": {
        "operation": null,
        "request": "f778c1b7c41ef175"
    },
    "environment": "westus2",
    "location": "westus2",
    "time": "2020-02-24T01:25:08.642942Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt\",\n        \"details\": []\n    },\n    \"correlation\": {\n        \"operation\": null,\n        \"request\": \"f778c1b7c41ef175\"\n    },\n    \"environment\": \"westus2\",\n    \"location\": \"westus2\",\n    \"time\": \"2020-02-24T01:25:08.642942Z\"\n}"
    }
}

In [14]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        with self.dataset.mount() as mount_context:
            self.imgs = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PNGImages"))))
            self.masks = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        with self.dataset.mount() as mount_context:
            img_path = os.path.join(mount_context.mount_point, "PNGImages", self.imgs[idx])
            mask_path = os.path.join(mount_context.mount_point, "PedMasks", self.masks[idx])
            img = Image.open(img_path).convert("RGB")
            # note that we haven't converted the mask to RGB,
            # because each color corresponds to a different instance
            # with 0 being background
            mask = Image.open(mask_path)

            mask = np.array(mask)
            # instances are encoded as different colors
            obj_ids = np.unique(mask)
            # first id is the background, so remove it
            obj_ids = obj_ids[1:]

            # split the color-encoded mask into a set
            # of binary masks
            masks = mask == obj_ids[:, None, None]

            # get bounding box coordinates for each mask
            num_objs = len(obj_ids)
            boxes = []
            for i in range(num_objs):
                pos = np.where(masks[i])
                xmin = np.min(pos[1])
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
                boxes.append([xmin, ymin, xmax, ymax])

            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            # there is only one class
            labels = torch.ones((num_objs,), dtype=torch.int64)
            masks = torch.as_tensor(masks, dtype=torch.uint8)

            image_id = torch.tensor([idx])
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            # suppose all instances are not crowd
            iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

            target = {}
            target["boxes"] = boxes
            target["labels"] = labels
            target["masks"] = masks
            target["image_id"] = image_id
            target["area"] = area
            target["iscrowd"] = iscrowd

            if self.transforms is not None:
                img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [16]:
!pip install --upgrade torchvision

Requirement already up-to-date: torchvision in /Users/gkv/miniconda3/envs/azureml/lib/python3.8/site-packages (0.5.0)


In [27]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [19]:
import sys
sys.path.insert(0,'./')

In [20]:
from platform import python_version

print(python_version())
help("modules")

3.8.1

Please wait a moment while I gather a list of all available modules...



/Users/gkv/miniconda3/envs/azureml/lib/python3.8/site-packages/IPython/kernel/__init__.py:12: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  warn("The `IPython.kernel` package has been deprecated since IPython 4.0."
/Users/gkv/miniconda3/envs/azureml/lib/python3.8/pkgutil.py:107: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, info.name+'.', onerror)


Cython              asynchat            inspect             random
IPython             asyncio             io                  re
OpenSSL             asyncore            ipaddress           readline
PIL                 atexit              ipykernel           reprlib
PyQt5               attr                ipykernel_launcher  requests
__future__          audioop             ipython_genutils    requests_oauthlib
_abc                automl              ipywidgets          resource
_ast                autoreload          isodate             rlcompleter
_asyncio            azureml             itertools           rmagic
_bisect             backcall            jaraco              runpy
_blake2             backports           jedi                sched
_bootlocale         base64              jeepney             secrets
_bz2                bdb                 jinja2              secretstorage
_cffi_backend       binascii            jmespath            select
_codecs             binhex           

/Users/gkv/miniconda3/envs/azureml/lib/python3.8/pkgutil.py:92: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  __import__(info.name)


In [28]:
# from engine import train_one_epoch, evaluate
# import utils
# import transforms as T

# def get_transform(train):
#     transforms = []
#     # converts the image, a PIL image, into a PyTorch Tensor
#     transforms.append(T.ToTensor())
#     if train:
#         # during training, randomly flip the training images
#         # and ground-truth for data augmentation
#         transforms.append(T.RandomHorizontalFlip(0.5))
#     return T.Compose(transforms)


In [33]:
# # use our dataset and defined transformations
# dataset = PennFudanDataset(penn_ds, get_transform(train=True))
# dataset_test = PennFudanDataset(penn_ds, get_transform(train=False))

# # split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# # define training and validation data loaders
# data_loader = torch.utils.data.DataLoader(
#     dataset, batch_size=2, shuffle=True, num_workers=4,
#     collate_fn=utils.collate_fn)

# data_loader_test = torch.utils.data.DataLoader(
#     dataset_test, batch_size=1, shuffle=False, num_workers=4,
#     collate_fn=utils.collate_fn)

In [34]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # our dataset has two classes only - background and person
# num_classes = 2

# # get the model using our helper function
# model = get_instance_segmentation_model(num_classes)
# # move model to the right device
# model.to(device)

# # construct an optimizer
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)

# # and a learning rate scheduler which decreases the learning rate by
# # 10x every 3 epochs
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                                step_size=3,
#                                                gamma=0.1)

In [1]:
# num_epochs = 10

# for epoch in range(num_epochs):
#     # train for one epoch, printing every 10 iterations
#     train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
#     # update the learning rate
#     lr_scheduler.step()
#     # evaluate on the test dataset
#     evaluate(model, data_loader_test, device=device)

NameError: name 'train_one_epoch' is not defined

In [ ]:
# # pick one image from the test set
# img, _ = dataset_test[0]
# # put the model in evaluation mode
# model.eval()
# with torch.no_grad():
#     prediction = model([img.to(device)])

In [ ]:
# Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())